<a href="https://colab.research.google.com/github/Diggi14/project_Property2/blob/main/Data_filling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Diggi14/project_Property2.git

Cloning into 'project_Property2'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 19 (delta 5), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 1.02 MiB | 6.67 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [3]:
df=pd.read_csv('/content/project_Property2/gurgaon_cleaned.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9396 entries, 0 to 9395
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           9396 non-null   int64  
 1   PREFERENCE           9396 non-null   object 
 2   PROPERTY_TYPE        9396 non-null   object 
 3   BEDROOM_NUM          9392 non-null   float64
 4   BATHROOM_NUM         9396 non-null   int64  
 5   BALCONY_NUM          9382 non-null   float64
 6   PRICE_PER_UNIT_AREA  9396 non-null   float64
 7   FURNISH              9103 non-null   object 
 8   AGE                  9394 non-null   object 
 9   FLOOR_NUM            9396 non-null   float64
 10  TOTAL_FLOOR          9396 non-null   float64
 11  SUPERBUILTUP_SQFT    9396 non-null   float64
 12  PRICE                9396 non-null   float64
 13  LOCALITY_WO_CITY     9396 non-null   object 
 14  stories              9396 non-null   object 
dtypes: float64(7), int64(2), object(6)
mem

In [5]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [6]:
df.head(4)

,PREFERENCE,PROPERTY_TYPE,BEDROOM_NUM,BATHROOM_NUM,BALCONY_NUM,PRICE_PER_UNIT_AREA,FURNISH,AGE,FLOOR_NUM,TOTAL_FLOOR,SUPERBUILTUP_SQFT,PRICE,LOCALITY_WO_CITY,stories
0,S,Residential Apartment,4.0,4,4.0,8766.0,Semi-furnished,5-10,14.0,26.0,3434.0,2.63,Sector 84,single
1,S,Residential Apartment,4.0,4,3.0,21176.0,Semi-furnished,1-5,7.0,29.0,2870.0,3.60,Sector 81,single
2,S,Residential Apartment,3.0,3,3.0,13740.0,Semi-furnished,1-5,14.0,28.0,2802.0,3.85,Sector 112,single
3,S,Residential Apartment,3.0,4,4.0,8515.0,Semi-furnished,1-5,4.0,27.0,2290.0,1.95,Sector 104,single


In [7]:
x=df.drop('PRICE',axis=1)
y=df['PRICE']

In [8]:
num_cols = ['BEDROOM_NUM','BATHROOM_NUM','PRICE_PER_UNIT_AREA','FLOOR_NUM','TOTAL_FLOOR','SUPERBUILTUP_SQFT']
cat_cols = ['PREFERENCE','PROPERTY_TYPE','FURNISH','AGE','LOCALITY_WO_CITY','stories','BALCONY_NUM']

num_pipeline = Pipeline([
    ('imputer', KNNImputer()),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

param_grid = {
    'preprocessor__num__imputer__n_neighbors': [2, 3, 5, 7, 10]
}

grid = GridSearchCV(pipeline, param_grid, cv=5)
grid.fit(x, y)

print("Best n_neighbors:", grid.best_params_)

Best n_neighbors: {'preprocessor__num__imputer__n_neighbors': 10}


In [9]:
df_numeric = df.select_dtypes(include=['number'])

imputer = KNNImputer(n_neighbors=10)
df_numeric = pd.DataFrame(
    imputer.fit_transform(df_numeric),
    columns=df_numeric.columns
)

df[df_numeric.columns] = df_numeric


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9396 entries, 0 to 9395
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PREFERENCE           9396 non-null   object 
 1   PROPERTY_TYPE        9396 non-null   object 
 2   BEDROOM_NUM          9396 non-null   float64
 3   BATHROOM_NUM         9396 non-null   float64
 4   BALCONY_NUM          9396 non-null   float64
 5   PRICE_PER_UNIT_AREA  9396 non-null   float64
 6   FURNISH              9103 non-null   object 
 7   AGE                  9394 non-null   object 
 8   FLOOR_NUM            9396 non-null   float64
 9   TOTAL_FLOOR          9396 non-null   float64
 10  SUPERBUILTUP_SQFT    9396 non-null   float64
 11  PRICE                9396 non-null   float64
 12  LOCALITY_WO_CITY     9396 non-null   object 
 13  stories              9396 non-null   object 
dtypes: float64(8), object(6)
memory usage: 1.0+ MB


In [11]:
df['FURNISH'] = df['FURNISH'].fillna(df['FURNISH'].mode()[0])

In [12]:
imputer = SimpleImputer(strategy='most_frequent')

df[['AGE']] = imputer.fit_transform(df[['AGE']])

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9396 entries, 0 to 9395
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PREFERENCE           9396 non-null   object 
 1   PROPERTY_TYPE        9396 non-null   object 
 2   BEDROOM_NUM          9396 non-null   float64
 3   BATHROOM_NUM         9396 non-null   float64
 4   BALCONY_NUM          9396 non-null   float64
 5   PRICE_PER_UNIT_AREA  9396 non-null   float64
 6   FURNISH              9396 non-null   object 
 7   AGE                  9396 non-null   object 
 8   FLOOR_NUM            9396 non-null   float64
 9   TOTAL_FLOOR          9396 non-null   float64
 10  SUPERBUILTUP_SQFT    9396 non-null   float64
 11  PRICE                9396 non-null   float64
 12  LOCALITY_WO_CITY     9396 non-null   object 
 13  stories              9396 non-null   object 
dtypes: float64(8), object(6)
memory usage: 1.0+ MB


In [14]:
df.to_csv('cleaned_data2.csv', index=False)